In [2]:
import os
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from timm import create_model
from itertools import product
from sklearn.metrics import roc_auc_score, confusion_matrix, roc_curve
from torchvision.models import resnet50
import matplotlib.pyplot as plt
import random
import time
import warnings
warnings.filterwarnings("ignore")

In [3]:
class LensingDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = ['no', 'sphere', 'vort']
        self.files = []
        for idx, cls in enumerate(self.classes):
            cls_dir = os.path.join(root_dir, cls)
            for f in os.listdir(cls_dir):
                if f.endswith('.npy'):
                    self.files.append((os.path.join(cls_dir, f), idx))

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        file_path, label = self.files[idx]
        img = np.load(file_path)  # (1, 150, 150), float64, 0-1
        img = torch.tensor(img, dtype=torch.float32)
        if self.transform:
            img = self.transform(img)
        return img, label

In [4]:
# Modified the ResNet50 for 1-channel input
class CustomResNet50(nn.Module):
    def __init__(self, num_classes=3, dropout_rate=0.0):
        super().__init__()
        base_model = resnet50(pretrained=True)
        # Replace first conv layer (3 channels -> 1 channel)
        self.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        # Copy weights for single channel (mean across RGB)
        with torch.no_grad():
            self.conv1.weight = nn.Parameter(base_model.conv1.weight.mean(dim=1, keepdim=True))
        # Rest of ResNet
        self.bn1 = base_model.bn1
        self.relu = base_model.relu
        self.maxpool = base_model.maxpool
        self.layer1 = base_model.layer1
        self.layer2 = base_model.layer2
        self.layer3 = base_model.layer3
        self.layer4 = base_model.layer4
        self.avgpool = base_model.avgpool
        self.fc = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(base_model.fc.in_features, num_classes)
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

In [5]:
# Resized the 150 x 150 images into 224 x 224 (supporting Resnet inputs)
# Applied Data Augmentation to only training data
# The images are already min-max normalized (0 to 1), slapping on extra normalization would be an overkill - skipped it.
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomAffine(degrees=15, translate=(0.1, 0.1))
])
val_transform = transforms.Compose([
    transforms.Resize((224, 224))
])

In [6]:
train_dataset = LensingDataset('/kaggle/input/deeplense-common-test/dataset/train', transform=train_transform)
val_dataset = LensingDataset('/kaggle/input/deeplense-common-test/dataset/train', transform=val_transform)

In [7]:
learning_rates = [1e-5, 3e-5, 1e-4, 3e-4]
batch_sizes = [16, 32, 64]
drop_rates = [0.0, 0.1, 0.2]

In [8]:
# Initially tried to run a grid search over 36 combos (4 LRs × 3 BSs × 3 DRs), training each for 5 epochs to gauge val loss.
# Due to Time and Computational constrains, switched to Random search over 10 combos (not 36), 3 epochs each (not 5).

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
best_val_loss = float('inf')
best_params = None
trials = 10
start_tuning = time.time()  

for _ in range(trials):
    lr = random.choice(learning_rates)
    bs = random.choice(batch_sizes)
    dr = random.choice(drop_rates)
    print(f'\nTuning: LR={lr}, BS={bs}, DR={dr}')
    train_loader = DataLoader(train_dataset, batch_size=bs, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=bs, shuffle=False, num_workers=4)

    model = CustomResNet50(num_classes=3, dropout_rate=dr).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=3)

    for epoch in range(3):
        model.train()
        train_loss = 0
        for imgs, labels in train_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_loss /= len(train_loader)

        model.eval()
        val_loss = 0
        with torch.no_grad():
            for imgs, labels in val_loader:
                imgs, labels = imgs.to(device), labels.to(device)
                outputs = model(imgs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
        val_loss /= len(val_loader)

        print(f'Epoch {epoch+1}/3, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')
        scheduler.step()

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_params = (lr, bs, dr)


end_tuning = time.time()  
tuning_time = end_tuning - start_tuning
print(f'\nBest Params: LR={best_params[0]}, BS={best_params[1]}, DR={best_params[2]}, Val Loss={best_val_loss:.4f}')
print(f'Tuning completed in {tuning_time:.2f} seconds ({tuning_time/60:.2f} minutes)\n')



Tuning: LR=0.0003, BS=32, DR=0.2


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 169MB/s] 


Epoch 1/3, Train Loss: 1.0567, Val Loss: 0.9540
Epoch 2/3, Train Loss: 0.5756, Val Loss: 0.4656
Epoch 3/3, Train Loss: 0.3458, Val Loss: 0.2602

Tuning: LR=0.0001, BS=32, DR=0.0
Epoch 1/3, Train Loss: 0.7702, Val Loss: 0.3861
Epoch 2/3, Train Loss: 0.3848, Val Loss: 0.3038
Epoch 3/3, Train Loss: 0.2726, Val Loss: 0.1973

Tuning: LR=0.0001, BS=32, DR=0.2
Epoch 1/3, Train Loss: 0.7825, Val Loss: 0.4112
Epoch 2/3, Train Loss: 0.3905, Val Loss: 0.3116
Epoch 3/3, Train Loss: 0.2819, Val Loss: 0.1990

Tuning: LR=0.0003, BS=64, DR=0.2
Epoch 1/3, Train Loss: 1.0533, Val Loss: 0.9146
Epoch 2/3, Train Loss: 0.5324, Val Loss: 0.5246
Epoch 3/3, Train Loss: 0.3269, Val Loss: 0.2466

Tuning: LR=0.0001, BS=16, DR=0.2
Epoch 1/3, Train Loss: 0.8888, Val Loss: 0.4484
Epoch 2/3, Train Loss: 0.4407, Val Loss: 0.3063
Epoch 3/3, Train Loss: 0.2925, Val Loss: 0.2256

Tuning: LR=1e-05, BS=64, DR=0.1
Epoch 1/3, Train Loss: 1.0826, Val Loss: 1.0466
Epoch 2/3, Train Loss: 0.9792, Val Loss: 0.9097
Epoch 3/3, Trai

In [9]:
train_loader = DataLoader(train_dataset, batch_size=best_params[1], shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=best_params[1], shuffle=False, num_workers=4)
model = CustomResNet50(num_classes=3, dropout_rate=best_params[2]).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=best_params[0])
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=15)

start_training = time.time()  

for epoch in range(15):
    model.train()
    train_loss = 0
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss /= len(train_loader)

    model.eval()
    val_loss = 0
    all_preds, all_labels = [], []
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            preds = torch.softmax(outputs, dim=1)
            all_preds.append(preds.cpu().numpy())
            all_labels.append(labels.cpu().numpy())
    val_loss /= len(val_loader)

    print(f'Final Epoch {epoch+1}/15, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')
    scheduler.step

end_training = time.time()  
training_time = end_training - start_training

print(f'Training completed in {training_time:.2f} seconds ({training_time/60:.2f} minutes)\n')

Final Epoch 1/15, Train Loss: 0.8281, Val Loss: 0.5354
Final Epoch 2/15, Train Loss: 0.4250, Val Loss: 0.3714
Final Epoch 3/15, Train Loss: 0.3402, Val Loss: 0.2644
Final Epoch 4/15, Train Loss: 0.2930, Val Loss: 0.2392
Final Epoch 5/15, Train Loss: 0.2675, Val Loss: 0.2613
Final Epoch 6/15, Train Loss: 0.2442, Val Loss: 0.2705
Final Epoch 7/15, Train Loss: 0.2316, Val Loss: 0.1764
Final Epoch 8/15, Train Loss: 0.2205, Val Loss: 0.1647
Final Epoch 9/15, Train Loss: 0.2084, Val Loss: 0.1410
Final Epoch 10/15, Train Loss: 0.1975, Val Loss: 0.1566
Final Epoch 11/15, Train Loss: 0.1935, Val Loss: 0.1374
Final Epoch 12/15, Train Loss: 0.1841, Val Loss: 0.1320
Final Epoch 13/15, Train Loss: 0.1787, Val Loss: 0.1348
Final Epoch 14/15, Train Loss: 0.1705, Val Loss: 0.1288
Final Epoch 15/15, Train Loss: 0.1668, Val Loss: 0.1535
Training completed in 5724.37 seconds (95.41 minutes)



In [10]:
torch.save(model.state_dict(), "resnet_model.pth")
print("Model saved as resnet_model.pth")

Model saved as resnet_model.pth


In [13]:
train_losses = [0.8281, 0.4250, 0.3402, 0.2930, 0.2675, 0.2442, 0.2316, 0.2205, 0.2084, 0.1975, 0.1935, 0.1841, 0.1787, 0.1705, 0.1668]
val_losses = [0.5354, 0.3714, 0.2644, 0.2392, 0.2613, 0.2705, 0.1764, 0.1647, 0.1410, 0.1566, 0.1374, 0.1320, 0.1348, 0.1288, 0.1535]
plt.figure(figsize=(8, 6))
plt.plot(range(1, 16), train_losses, label="Train Loss", marker="o")
plt.plot(range(1, 16), val_losses, label="Validation Loss", marker="s")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.title("Loss Curve")
plt.grid(True)
plt.savefig("loss_curve.png", dpi=300)
plt.close()
plt.show()

In [14]:
all_preds = np.concatenate(all_preds)
all_labels = np.concatenate(all_labels)
class_names = ['no', 'sphere', 'vort']

In [15]:
auc_scores = [roc_auc_score(all_labels == i, all_preds[:, i]) for i in range(3)]
print(f'AUC Scores: {class_names[0]}={auc_scores[0]:.3f}, {class_names[1]}={auc_scores[1]:.3f}, {class_names[2]}={auc_scores[2]:.3f}, Overall={np.mean(auc_scores):.3f}')

AUC Scores: no=0.995, sphere=0.992, vort=0.997, Overall=0.995


In [16]:
plt.figure(figsize=(8, 6))
for i in range(3):
    fpr, tpr, _ = roc_curve(all_labels == i, all_preds[:, i])
    plt.plot(fpr, tpr, label=f'{class_names[i]} (AUC = {auc_scores[i]:.3f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves (ResNet50, 224x224)')
plt.legend()
plt.grid(True)
plt.savefig('roc_curves_resnet.png')
plt.close()

In [17]:
cm = confusion_matrix(all_labels, all_preds.argmax(axis=1))
print('Confusion Matrix:')
print(cm)

Confusion Matrix:
[[9992    3    5]
 [ 813 9013  174]
 [ 401  101 9498]]


In [18]:
plt.figure(figsize=(6, 6))
plt.imshow(cm, interpolation='nearest', cmap='Blues')
plt.title('Confusion Matrix (ResNet50, 224x224)')
plt.colorbar()
tick_marks = np.arange(3)
plt.xticks(tick_marks, class_names, rotation=45)
plt.yticks(tick_marks, class_names)
for i in range(3):
    for j in range(3):
        plt.text(j, i, cm[i, j], ha='center', va='center', color='black' if cm[i, j] < cm.max() / 2 else 'white')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.tight_layout()
plt.savefig('confusion_matrix_resnet.png')
plt.close()

In [19]:
print("Plots saved as 'roc_curves_resnet.png' and 'confusion_matrix_resnet.png'")

Plots saved as 'roc_curves_resnet.png' and 'confusion_matrix_resnet.png'
